In [3]:
import os
from os import listdir
from PIL import Image
import pandas as pd
import PIL
from sklearn.utils import shuffle
import numpy as np
import cv2
import skimage
from skimage.feature import graycomatrix,graycoprops

In [4]:
directory_root = "../input/brain-new-dataset/Dataset_new"
print(os.listdir("../input/brain-new-dataset/Dataset_new"))
print(len(listdir(directory_root)))

['Isc', 'H1']
2


In [5]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for stroke_folder in root_dir :
        stroke_type_folder_list = listdir(f"{directory_root}/{stroke_folder}")
        print(f"Processing {stroke_folder} ...")
      

        for image in stroke_type_folder_list:
            image_directory = f"{directory_root}/{stroke_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(image_directory)
                label_list.append(stroke_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
Processing Isc ...
Processing H1 ...
[INFO] Image loading completed


In [6]:
img_data = pd.DataFrame({'ImagePath':image_list,'Label':label_list})
print(img_data.head())
print(len(img_data))

                                           ImagePath Label
0  ../input/brain-new-dataset/Dataset_new/Isc/Isc...   Isc
1  ../input/brain-new-dataset/Dataset_new/Isc/Isc...   Isc
2  ../input/brain-new-dataset/Dataset_new/Isc/Isc...   Isc
3  ../input/brain-new-dataset/Dataset_new/Isc/Isc...   Isc
4  ../input/brain-new-dataset/Dataset_new/Isc/Isc...   Isc
2000


In [7]:
img_data["LabelsInteger"] = None

index_labels_integer = img_data.columns.get_loc("LabelsInteger")

index_type = img_data.columns.get_loc("Label")

j = 0 
for i in range(len(img_data)):
    if i == 0:
        img_data.iloc[i, index_labels_integer] = j
    if i > 0:
        if img_data.iloc[i-1, index_type] == img_data.iloc[i, index_type]:
            img_data.iloc[i, index_labels_integer] = j
        else:
            j += 1
            img_data.iloc[i, index_labels_integer] = j

In [8]:
img_data.drop_duplicates(subset = ["LabelsInteger"])

,ImagePath,Label,LabelsInteger
0,../input/brain-new-dataset/Dataset_new/Isc/Isc...,Isc,0
1000,../input/brain-new-dataset/Dataset_new/H1/Heam...,H1,1


In [9]:
img_data= shuffle(img_data)
labels=img_data["LabelsInteger"]
contrast,energy,homogeneity,correlation, dissimilarity,ASM=[],[],[],[],[],[]

for image_path in img_data.ImagePath:
    img = Image.open(image_path)
    size = (256, 256)
    img = img.resize(size, PIL.Image.ANTIALIAS) 
    img=img.convert('L')
    im = skimage.img_as_ubyte(img)
    im =im// 32
    glcm=skimage.feature.graycomatrix(im, [1], [135], levels=8)
    contrast.append(skimage.feature.graycoprops(glcm, 'contrast')[0][0])
    energy.append(skimage.feature.graycoprops(glcm, 'energy')[0][0])
    homogeneity.append(skimage.feature.graycoprops(glcm, 'homogeneity')[0][0])
    correlation.append(skimage.feature.graycoprops(glcm, 'correlation')[0][0])
    dissimilarity.append(skimage.feature.graycoprops(glcm, 'dissimilarity')[0][0])
    ASM.append(skimage.feature.graycoprops(glcm, 'ASM')[0][0])
    
   
    
    

In [10]:
features = pd.DataFrame({'contrast':contrast,'energy':energy,'homogeneity':homogeneity,'correlation':correlation,'dissimilarity':dissimilarity,'ASM':ASM,'Type':labels})
features
#print(len(features))

,contrast,energy,homogeneity,correlation,dissimilarity,ASM,Type
1535,0.265104,0.681976,0.951974,0.969185,0.121415,0.465091,1
856,0.169975,0.665989,0.951879,0.978554,0.108303,0.443541,0
1592,0.231878,0.544150,0.933016,0.973717,0.149648,0.296099,1
165,0.365074,0.370526,0.891452,0.972098,0.240104,0.137289,0
771,0.336826,0.401154,0.908950,0.961994,0.205086,0.160925,0
...,...,...,...,...,...,...,...
1637,0.283931,0.530348,0.922029,0.969355,0.176823,0.281269,1
732,0.298039,0.585320,0.938341,0.965773,0.149510,0.342600,0
835,0.304274,0.388590,0.905717,0.972117,0.207430,0.151002,0
1490,0.222181,0.734091,0.964740,0.960782,0.092096,0.538889,1


In [11]:
features.to_csv('features.csv')